In [4]:
import json
import pandas as pd

/tmp/ipykernel_95727/748694261.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Etcd latency generation block
def parse_log_data(log_file):
    with open(log_file, 'r') as f:
        json_data = [json.loads(line) for line in f if line.strip()]
    return json_data

def etcd_data_proc(log_file):
    log_data = parse_log_data(log_file)
    df = pd.DataFrame(log_data)
    unique_ksizes = df['KeySize'].unique().tolist()
    print(unique_ksizes)
    put_events = df[df['Event'] == 'PUT']
    trigger_events = df[df['Event'] == 'TRIGGER']
    # print(put_events)
    
    res = {}
    for ksize in unique_ksizes:
        put_events = df[(df['Event'] == 'PUT') & (df['KeySize'] == ksize)]
        trigger_events = df[(df['Event'] == 'TRIGGER') & (df['KeySize'] == ksize)]
        
        put_events = put_events.set_index(['Key', 'KeyVersion'])
        # print(put_events)
        trigger_events = trigger_events.set_index(['Key', 'KeyVersion'])
        merged_events = put_events.join(trigger_events, lsuffix='_PUT', rsuffix='_TRIGGER', how='inner')
        # print(merged_events)
        merged_events['Timestamp_Difference'] = 1000*(merged_events['time_stamp_TRIGGER'] - merged_events['time_stamp_PUT'])
        
        res[ksize] = merged_events['Timestamp_Difference'].tolist()
    print(res)
    return res

latencies_dict = etcd_data_proc('etcd_logging.log')


In [ ]:
# Dynamo latency generation block

In [ ]:
# S3 & S3Express latency generation block

In [ ]:
# Combine latency data

In [ ]:
# Graph gen block